<style>
p{
    line-height:1.5;
}
</style>
## Behandling av værdata
I denne filen behandler og manipulerer vi dataen slik at den blir lettere å lese og visualisere – den forberedes for videre analyse. 

In [5]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import json

load_dotenv()

# Henter API-key/Client-id fra .env filen
client_id = os.getenv("client_id")

with open('../data/Byer.json', 'r') as f:
    byer = json.load(f)

byerAsString = ','.join(byer.values())

element_id = "mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)"  # Temperatur, nedbør og vindhastighet
start_dato = "2022-01-01"
slutt_dato = "2023-01-01" 

# Definerer paramtere
endpoint = 'https://frost.met.no/observations/v0.jsonld'
parameters = {
    "sources": byerAsString,
    "elements": element_id,
    "referencetime": f"{start_dato}/{slutt_dato}",
    "timeoffsets": "default",
    "timeresolutions": "P1D",
    "levels": "default"
}
print(f"Parametere: {parameters}")

# Sender forespørsel til Frost API
response = requests.get(endpoint, params=parameters, auth=(client_id, ''))

# Sjekker om forespørselen var vellykket
if response.status_code != 200:
    print(f"Error! Status code: {response.status_code}")
    print(f"Message: {response.json().get('error', {}).get('message', 'Ingen melding gitt')}")
    print(f"Reason: {response.json().get('error', {}).get('reason', 'Ingen årsak gitt')}")
    exit()

# Henter JSON-data
json_data = response.json()
if 'data' not in json_data:
    print("Ingen data funnet i forespørselen.")
    print(json_data)
    exit()

# Prosesserer data til en DataFrame
data = json_data['data']
rows = []
for item in data:
    for observation in item['observations']:
        row = {
            'sourceId': item['sourceId'].split(':')[0],
            'referenceTime': item['referenceTime'],
            'elementId': observation['elementId'],
            'value': observation['value'],
            'unit': observation['unit'],
            'codequality': observation.get('qualityCode', '')  # Henter kvalitetskoden
        }
        rows.append(row)

# Opprett DataFrame fra rader
df = pd.DataFrame(rows)

# Legger til en ny kolonne for bynavn basert på sourceId
df['by'] = df['sourceId'].map({v: k for k, v in byer.items()})

# Forenkler tidsformatet (YYYY-MM-DD)
df['referenceTime'] = pd.to_datetime(df['referenceTime']).dt.date

# Splitter "elementId" i flere kolonner
df[['statistikk', 'variable']] = df['elementId'].str.extract(r'(\w+)\(([^)]+)')

# Gjør "codequality"-verdiene mer forståelige ved bruk av list comprehensions
df['codequality'] = ["God" if x in [0, 1, 2] else "Middels" if x in [3, 4, 5] else "Dårlig" for x in df['codequality']]

# Velger ønskede kolonner
df = df[['by', 'sourceId', 'referenceTime', 'statistikk', 'variable', 'value', 'unit', 'codequality']]

# Skriver DataFrame til en CSV-fil
data_fil = '../data/BehandletVaerData.csv'
df.to_csv(data_fil, index=False)
print(f"Data lagret i {data_fil}")

Parametere: {'sources': 'SN68230,SN18700,SN50540', 'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)', 'referencetime': '2022-01-01/2023-01-01', 'timeoffsets': 'default', 'timeresolutions': 'P1D', 'levels': 'default'}
Data lagret i ../data/BehandletVaerData.csv


<style>
p{
    line-height:1.5;
}
</style>
Her har vi gjort om kodekvaliteten fra tilfeldige tall til mer beskrivende tekst ved bruk av list comprehensions. I steden for at kodekvaliteten har et tall mellom 0 og 7, har vi endret det slik at kodekvalitet 0-2 skrives som "God", kodekvalitet 3-5 skrives som "Middels" og kodekvalitet 6 og 7 skrives som "Dårlig". 

In [6]:
import pandas as pd

# Last inn CSV-filen
data_fil = '../data/VaerData.csv' 
df = pd.read_csv(data_fil)

# Sjekk for manglende verdier
print("Antall manglende verdier per kolonne:")
print(df.isnull().sum())

# Finner rader med manglende verdier
print("\nRader med manglende verdier:")
print(df[df.isnull().any(axis=1)].index + 2)

Antall manglende verdier per kolonne:
by                0
sourceId          0
referenceTime     0
statistikk        0
variable          0
value             0
unit              0
codequality      10
dtype: int64

Rader med manglende verdier:
Index([287, 320, 1281, 1382, 1415, 1431, 1437, 1440, 1626, 2382], dtype='int64')


<style>
p{
    line-height:1.5;
}
</style>
Her bruker vi Pandas til å identifisere manglende verdier i rådataen. Vi ser at det kun er i kodekvalitet-kolonnen at det er manglende verdier. Det er også listet opp hvilke rader som har manglende verdier. 

In [7]:
import pandas as pd

# Laster inn CSV-filen
data_fil = '../data/BehandletVaerData.csv' 
df = pd.read_csv(data_fil)

# Sjekker for manglende verdier
print("Manglende verdier per kolonne:")
print(df.isnull().sum())

# Fyller inn manglende 'codequality' med den vanligste verdien
vanligste_verdi = df['codequality'].mode()[0]  # Finner vanligste verdi
df = df.assign(codequality=df['codequality'].fillna(vanligste_verdi))
print(f"\nManglende 'codequality'-verdier fylt med: {vanligste_verdi}")

# Sjekker på nytt for å bekrefte at det ikke er flere manglende verdier
print("\nEtter utfylling av manglende verdier:")
print(df.isnull().sum())

# Lagrer den oppdaterte DataFrame-en i CSV-filen
data_fil = '../data/BehandletVaerData.csv'
df.to_csv(data_fil, index=False)
print(f"\nOppdatert data lagret i {data_fil}")

# Rader med manglende verdier:
print("\nRader med manglende verdier:")
print(df[df.isnull().any(axis=1)].index + 2)

Manglende verdier per kolonne:
by               0
sourceId         0
referenceTime    0
statistikk       0
variable         0
value            0
unit             0
codequality      0
dtype: int64

Manglende 'codequality'-verdier fylt med: God

Etter utfylling av manglende verdier:
by               0
sourceId         0
referenceTime    0
statistikk       0
variable         0
value            0
unit             0
codequality      0
dtype: int64

Oppdatert data lagret i ../data/BehandletVaerData.csv

Rader med manglende verdier:
Index([], dtype='int64')


<style>
p{
    line-height:1.5;
}
</style>
Etter vi analyserte og identifiserte hvilke rader som hadde manglende verdier, fylte vi inn verider der det opprinnelig manglet. Vi fant ut av hvilken kodekvalitet som var den hyppigste, og fylte inn den der det var manglende verdier. 

In [31]:
# Sjekker hvilke dager som har temperaturer under 0 grader
dager_negativ_temp = [row for index, row in df.iterrows() if row['value'] < 0 and row['variable'] == "air_temperature P1D"]

print(f"Antall dager mellom {start_dato} og {slutt_dato} med temperatur under 0 grader: {len(dager_negativ_temp)}")

Antall dager mellom 2022-01-01 og 2023-01-01 med temperatur under 0 grader: 140


<div style="border:2px solid yellow; padding:10px;border-radius:5px;">
<style>
p{
    line-height:1.5;
}
</style>
For å ha oversikt over antall dager med minusgrader vi har, brukte vi her list comprehensions til å finne ut av nettopp det. Det kan være nyttig informasjon å ha til videre analyse av værdataen. 
</div>

<div style="border:2px solid yellow; padding:10px;border-radius:5px;">
<style>
p{
    line-height:1.5;
}
</style>

## Uregelmessigheter i dataene og håndtering av dem
Vi forventer å møte på uregelmessigheter i dataene våre når vi jobber med dem fremover, og vi har også gjort det allerede. Vi har for eksempel erfart at kodekvaliteten var ganske varierende og uregelmessig, og det håndterte vi ved å endre dataen til et mer forståelig format og fylle inn for de manglende verdiene med den vanligste verdien. Det gjorde vi fordi det statistisk sett er mer sannsynlig at den hyppigste verdien skal være der det er manglende verdier enn noen andre. 

I tillegg til manglende verdier er det sannsynlig at vi kan møte på f.eks. feilaktige verdier og feilaktige enheter. Den overordnede planen vår for håndtering av uregelmessigheter er at vi må identifisere problemet, deretter rense og formatere dataene og til slutt loggføre uregelmessighetene. Å loggføre uregelmessighetene kan være lurt for å ha en oversikt over hvilke endringer som er gjort med uregelmessig data. 
</div>